In [1]:
!pip install pymysql
!pip install pymssql

import LibData
import LibUtils
import LibML
import LibFile
import LibEE

import pandas as pd
import numpy as np


     |████████████████████████████████| 51kB 2.9MB/s 
     |████████████████████████████████| 1.3MB 6.2MB/s 


In [2]:
jobName = '201216_Train_split1'
# jobName = '201216_Train_pilot'

x, y = LibData.GetXY_o(jobName)
x = LibData.GetMoneyness(x)
x = LibData.GetIrReduction(x)


In [ ]:
############## Bef
scaler, x1 = LibUtils.ScaleStandard(x)
print('Bef RF/XGBoost')
print(LibML.GetScoresCV(x1,y,LibML.RandomForestDefault(len(x1.columns)), 5), LibML.GetScoresCV(x1,y,LibML.XGBoostDefault(), 5))
# [160.16, 0.1915, 0.9143], [151.04, 0.1806, 0.9455] 

In [3]:
############## 현재나이 테스트

x2 = LibData.GetAttainedAge(x)
scaler, x2 = LibUtils.ScaleStandard(x2)
print('Test AttainedAge RF/XGBoost')
print(LibML.GetScoresCV(x2,y,LibML.RandomForestDefault(len(x2.columns)), 5), LibML.GetScoresCV(x2,y,LibML.XGBoostDefault(), 5))
#  [148.33, 0.1774, 0.9281] [137.54, 0.1644, 0.9583]

Test AttainedAge RF/XGBoost
start learning 1 2021-01-17 14:01... [146.98, 0.1761, 0.9294]... end learning 2021-01-17 14:04
start learning 2 2021-01-17 14:04... [147.25, 0.1777, 0.9335]... end learning 2021-01-17 14:07
start learning 3 2021-01-17 14:07... [149.0, 0.1806, 0.926]... end learning 2021-01-17 14:10
start learning 4 2021-01-17 14:10... [148.3, 0.1762, 0.9313]... end learning 2021-01-17 14:13
start learning 5 2021-01-17 14:13... [152.7, 0.1793, 0.9172]... end learning 2021-01-17 14:16
start learning 1 2021-01-17 14:16... [138.97, 0.1657, 0.9571]... end learning 2021-01-17 14:20
start learning 2 2021-01-17 14:20... [138.23, 0.166, 0.9578]... end learning 2021-01-17 14:24
start learning 3 2021-01-17 14:24... [140.59, 0.1677, 0.9549]... end learning 2021-01-17 14:28
start learning 4 2021-01-17 14:28... [136.0, 0.1634, 0.9585]... end learning 2021-01-17 14:32
start learning 5 2021-01-17 14:32... [133.26, 0.1586, 0.9612]... end learning 2021-01-17 14:36
[148.85, 0.178, 0.9275] [137

In [4]:
############## Prem납입완료여부 테스트

def GetIsInPayment(xBase):
    x = xBase.copy(True)
    x['IsInPayment'] = x.apply(lambda row: 0 if row['ElapsedMth']//12 >= row['PremYr'] else 1, axis=1)
    return x

x2 = GetIsInPayment(x)
scaler, x2 = LibUtils.ScaleStandard(x2)
print('Test IsInPaymen RF/XGBoost')
print(LibML.GetScoresCV(x2,y,LibML.RandomForestDefault(len(x2.columns)), 5), LibML.GetScoresCV(x2,y,LibML.XGBoostDefault(), 5))
# [160.32, 0.1917, 0.9129] [150.37, 0.1798, 0.946] 

Test IsInPaymen RF/XGBoost
start learning 1 2021-01-17 14:36... [158.84, 0.1902, 0.914]... end learning 2021-01-17 14:39
start learning 2 2021-01-17 14:39... [161.08, 0.1931, 0.912]... end learning 2021-01-17 14:42
start learning 3 2021-01-17 14:42... [157.15, 0.1887, 0.921]... end learning 2021-01-17 14:45
start learning 4 2021-01-17 14:45... [161.8, 0.1933, 0.909]... end learning 2021-01-17 14:48
start learning 5 2021-01-17 14:48... [161.97, 0.1922, 0.9135]... end learning 2021-01-17 14:51
start learning 1 2021-01-17 14:51... [151.47, 0.1811, 0.9452]... end learning 2021-01-17 14:55
start learning 2 2021-01-17 14:55... [149.87, 0.1807, 0.9465]... end learning 2021-01-17 15:00
start learning 3 2021-01-17 15:00... [150.78, 0.1798, 0.943]... end learning 2021-01-17 15:04
start learning 4 2021-01-17 15:04... [148.97, 0.1783, 0.9457]... end learning 2021-01-17 15:08
start learning 5 2021-01-17 15:08... [151.37, 0.1798, 0.9466]... end learning 2021-01-17 15:12
[160.17, 0.1915, 0.9139] [150

In [ ]:
def GetMoneyness(x_):
    x_['Moneyness'] = x_.apply(lambda row: row['SumFund']/row['AccumPrem']*100 if row['AccumPrem'] > 0 else 0, axis=1)
    x_['FundVal01'] = x_.apply(lambda row: row['FundVal01']/row['SumFund']*100 if row['SumFund'] > 0 else 0, axis=1)
    x_['FundVal02'] = x_.apply(lambda row: row['FundVal02']/row['SumFund']*100 if row['SumFund'] > 0 else 0, axis=1)
    x_['FundVal03'] = x_.apply(lambda row: row['FundVal03']/row['SumFund']*100 if row['SumFund'] > 0 else 0, axis=1)
    del x_['SumFund']
    return x_

def GetAttainedAge(x_):
    x_['AttainedAge'] = x_.apply(lambda row: row['ContAge'] + row['ElapsedMth']//12, axis=1)
    del x_['ContAge']
    return x_


In [ ]:

jobName = '201216_Train_split1'
x, y = LibData.GetXYBase(jobName)
x = GetMoneyness(x)
x = GetAttainedAge(x)

x.to_csv('d:/x.csv', index=False)
y.to_csv('d:/y.csv', index=False)


In [ ]:
# 머니니스 만들고, 적립금컬럼 삭제, 펀드별금액을 비중으로 바꿈

x_ = x.copy(True)

x_['Moneyness'] = x_.apply(lambda row: row['SumFund']/row['AccumPrem']*100 if row['AccumPrem'] > 0 else 0, axis=1)
x_['FundVal01'] = x_.apply(lambda row: row['FundVal01']/row['SumFund']*100 if row['SumFund'] > 0 else 0, axis=1)
x_['FundVal02'] = x_.apply(lambda row: row['FundVal02']/row['SumFund']*100 if row['SumFund'] > 0 else 0, axis=1)
x_['FundVal03'] = x_.apply(lambda row: row['FundVal03']/row['SumFund']*100 if row['SumFund'] > 0 else 0, axis=1)
del x_['SumFund']

l_x, l_y = LibData.GetSplited(x,y,5)
l_x_, l_y_ = LibData.GetSplited(x_,y,5)

from sklearn.ensemble import RandomForestRegressor

# 기존 학습
model = RandomForestRegressor(n_estimators=700, max_features=26, min_samples_leaf=2)
# model.fit(xTrain, yTrain['pv_claim'])

tag = 'moneyness, pv_claim, tf' 

preds = LibML.GetPredCV(l_x, l_y, model, 'pv_claim')
maes = LibML.GetMaesCV(l_y, preds, 'pv_claim')
LibUtils.WriteNote(tag, 'base', [str(sum(maes)/5), maes])

preds_ = LibML.GetPredCV(l_x_, l_y_, model, 'pv_claim')
maes_ = LibML.GetMaesCV(l_y_, preds_, 'pv_claim')
LibUtils.WriteNote(tag, 'new', [str(sum(maes_)/5), maes_])

LibUtils.ELoop()

In [ ]:
# 현재나이 만듬

x_ = x.copy(True)
x_['AttainedAge'] = x_.apply(lambda row: row['ContAge'] + row['ElapsedMth']//12, axis=1)

l_x_, l_y_ = LibData.GetSplited(x_,y,5)

from sklearn.ensemble import RandomForestRegressor

model = RandomForestRegressor(n_estimators=700, max_features=26, min_samples_leaf=2)

tag = 'AttainedAge, pv_claim, tf' 
preds_ = LibML.GetPredCV(l_x_, l_y_, model, 'pv_claim')
maes_ = LibML.GetMaesCV(l_y_, preds_, 'pv_claim')
LibUtils.WriteNote(tag, 'create AA', [str(sum(maes_)/5), maes_])

###################################

# 현재나이 만듬 + 가입나이 삭제

x_ = x.copy(True)
x_['AttainedAge'] = x_.apply(lambda row: row['ContAge'] + row['ElapsedMth']//12, axis=1)
del x_['ContAge']

l_x_, l_y_ = LibData.GetSplited(x_,y,5)

from sklearn.ensemble import RandomForestRegressor

model = RandomForestRegressor(n_estimators=700, max_features=26, min_samples_leaf=2)

tag = 'AttainedAge, pv_claim, tf' 
preds_ = LibML.GetPredCV(l_x_, l_y_, model, 'pv_claim')
maes_ = LibML.GetMaesCV(l_y_, preds_, 'pv_claim')
LibUtils.WriteNote(tag, 'creating AA + Deleting CA', [str(sum(maes_)/5), maes_])

LibUtils.ELoop()

In [ ]:
# 동시반영 : 머니니스추가등 + 현재나이추가 + 가입나이삭제

x_ = x.copy(True)

x_['Moneyness'] = x_.apply(lambda row: row['SumFund']/row['AccumPrem']*100 if row['AccumPrem'] > 0 else 0, axis=1)
x_['FundVal01'] = x_.apply(lambda row: row['FundVal01']/row['SumFund']*100 if row['SumFund'] > 0 else 0, axis=1)
x_['FundVal02'] = x_.apply(lambda row: row['FundVal02']/row['SumFund']*100 if row['SumFund'] > 0 else 0, axis=1)
x_['FundVal03'] = x_.apply(lambda row: row['FundVal03']/row['SumFund']*100 if row['SumFund'] > 0 else 0, axis=1)
del x_['SumFund']

x_['AttainedAge'] = x_.apply(lambda row: row['ContAge'] + row['ElapsedMth']//12, axis=1)
del x_['ContAge']

from sklearn.ensemble import RandomForestRegressor

model = RandomForestRegressor(n_estimators=700, max_features=26, min_samples_leaf=2)

tag = 'both, pv_claim, tf' 
preds_ = LibML.GetPredCV(l_x_, l_y_, model, 'pv_claim')
maes_ = LibML.GetMaesCV(l_y_, preds_, 'pv_claim')
LibUtils.WriteNote(tag, 'moneyness+age', [str(sum(maes_)/5), maes_])

LibUtils.ELoop()

In [ ]:
# 예비코드
from sklearn.model_selection import train_test_split
xTrain, xVal, yTrain, yVal = train_test_split(x, y, test_size=0.2)

model.fit(xTrain, yTrain['pv_claim'])
y_pred = pd.Series(model.predict(xVal), index=xVal.index)

In [ ]:
######## 1_base, 2_moneyness, 3_현재나이, 4_moneyness+현재나이 테스트

from sklearn.ensemble import RandomForestRegressor
model = RandomForestRegressor(n_estimators=700, max_features=26, min_samples_leaf=2)


### 1_base
l_x, l_y = LibData.GetSplited(x,y,5)
preds = LibML.GetPredCV(l_x, l_y, model, 'pv_claim')
maes = LibML.GetMaesCV(l_y, preds, 'pv_claim')
print('1_base', [str(sum(maes)/5), maes])


### 2_머니니스
x_ = x.copy(True)
x_['Moneyness'] = x_.apply(lambda row: row['SumFund']/row['AccumPrem']*100 if row['AccumPrem'] > 0 else 0, axis=1)
x_['FundVal01'] = x_.apply(lambda row: row['FundVal01']/row['SumFund']*100 if row['SumFund'] > 0 else 0, axis=1)
x_['FundVal02'] = x_.apply(lambda row: row['FundVal02']/row['SumFund']*100 if row['SumFund'] > 0 else 0, axis=1)
x_['FundVal03'] = x_.apply(lambda row: row['FundVal03']/row['SumFund']*100 if row['SumFund'] > 0 else 0, axis=1)
del x_['SumFund']

l_x, l_y = LibData.GetSplited(x_, y, 5)
preds = LibML.GetPredCV(l_x, l_y, model, 'pv_claim')
maes = LibML.GetMaesCV(l_y, preds, 'pv_claim')
print('2_moneyness', [str(sum(maes)/5), maes])


### 3_현재나이
x_ = x.copy(True)
x_['AttainedAge'] = x_.apply(lambda row: row['ContAge'] + row['ElapsedMth']//12, axis=1)
#del x_['ContAge']

l_x, l_y = LibData.GetSplited(x_, y, 5)
preds = LibML.GetPredCV(l_x, l_y, model, 'pv_claim')
maes = LibML.GetMaesCV(l_y, preds, 'pv_claim')
print('3_AttainedAge', [str(sum(maes)/5), maes])


### 4_머니니스+현재나이
x_ = x.copy(True)
x_['Moneyness'] = x_.apply(lambda row: row['SumFund']/row['AccumPrem']*100 if row['AccumPrem'] > 0 else 0, axis=1)
x_['FundVal01'] = x_.apply(lambda row: row['FundVal01']/row['SumFund']*100 if row['SumFund'] > 0 else 0, axis=1)
x_['FundVal02'] = x_.apply(lambda row: row['FundVal02']/row['SumFund']*100 if row['SumFund'] > 0 else 0, axis=1)
x_['FundVal03'] = x_.apply(lambda row: row['FundVal03']/row['SumFund']*100 if row['SumFund'] > 0 else 0, axis=1)
del x_['SumFund']
x_['AttainedAge'] = x_.apply(lambda row: row['ContAge'] + row['ElapsedMth']//12, axis=1)
#del x_['ContAge']

l_x, l_y = LibData.GetSplited(x_, y, 5)
preds = LibML.GetPredCV(l_x, l_y, model, 'pv_claim')
maes = LibML.GetMaesCV(l_y, preds, 'pv_claim')
print('4_Moneyness+AttainedAge', [str(sum(maes)/5), maes])
      






######## 모델복잡도를 높여서 4가지 테스트
######## 1_base, 2_moneyness, 3_현재나이, 4_moneyness+현재나이 테스트

from sklearn.ensemble import RandomForestRegressor
model = RandomForestRegressor(n_estimators=1000, max_features=30, min_samples_leaf=1)


### 1_base
l_x, l_y = LibData.GetSplited(x,y,5)
preds = LibML.GetPredCV(l_x, l_y, model, 'pv_claim')
maes = LibML.GetMaesCV(l_y, preds, 'pv_claim')
print('1_base', [str(sum(maes)/5), maes])


### 2_머니니스
x_ = x.copy(True)
x_['Moneyness'] = x_.apply(lambda row: row['SumFund']/row['AccumPrem']*100 if row['AccumPrem'] > 0 else 0, axis=1)
x_['FundVal01'] = x_.apply(lambda row: row['FundVal01']/row['SumFund']*100 if row['SumFund'] > 0 else 0, axis=1)
x_['FundVal02'] = x_.apply(lambda row: row['FundVal02']/row['SumFund']*100 if row['SumFund'] > 0 else 0, axis=1)
x_['FundVal03'] = x_.apply(lambda row: row['FundVal03']/row['SumFund']*100 if row['SumFund'] > 0 else 0, axis=1)
del x_['SumFund']

l_x, l_y = LibData.GetSplited(x_, y, 5)
preds = LibML.GetPredCV(l_x, l_y, model, 'pv_claim')
maes = LibML.GetMaesCV(l_y, preds, 'pv_claim')
print('2_moneyness', [str(sum(maes)/5), maes])


### 3_현재나이
x_ = x.copy(True)
x_['AttainedAge'] = x_.apply(lambda row: row['ContAge'] + row['ElapsedMth']//12, axis=1)
#del x_['ContAge']

l_x, l_y = LibData.GetSplited(x_, y, 5)
preds = LibML.GetPredCV(l_x, l_y, model, 'pv_claim')
maes = LibML.GetMaesCV(l_y, preds, 'pv_claim')
print('3_AttainedAge', [str(sum(maes)/5), maes])


### 4_머니니스+현재나이
x_ = x.copy(True)
x_['Moneyness'] = x_.apply(lambda row: row['SumFund']/row['AccumPrem']*100 if row['AccumPrem'] > 0 else 0, axis=1)
x_['FundVal01'] = x_.apply(lambda row: row['FundVal01']/row['SumFund']*100 if row['SumFund'] > 0 else 0, axis=1)
x_['FundVal02'] = x_.apply(lambda row: row['FundVal02']/row['SumFund']*100 if row['SumFund'] > 0 else 0, axis=1)
x_['FundVal03'] = x_.apply(lambda row: row['FundVal03']/row['SumFund']*100 if row['SumFund'] > 0 else 0, axis=1)
del x_['SumFund']
x_['AttainedAge'] = x_.apply(lambda row: row['ContAge'] + row['ElapsedMth']//12, axis=1)
#del x_['ContAge']

l_x, l_y = LibData.GetSplited(x_, y, 5)
preds = LibML.GetPredCV(l_x, l_y, model, 'pv_claim')
maes = LibML.GetMaesCV(l_y, preds, 'pv_claim')
print('4_Moneyness+AttainedAge', [str(sum(maes)/5), maes])

LibUtils.ELoop()

start learning 1 2020-12-26 16:20... end learning 2020-12-26 16:28
start learning 2 2020-12-26 16:28... end learning 2020-12-26 16:36
start learning 3 2020-12-26 16:36... end learning 2020-12-26 16:45
start learning 4 2020-12-26 16:45... end learning 2020-12-26 16:53
start learning 5 2020-12-26 16:53... end learning 2020-12-26 17:01
1_base ['155.8104417496264', [156.93994273062205, 158.2932151665893, 158.90776386933226, 152.36568233156436, 152.54560465002396]]
start learning 1 2020-12-26 17:01... end learning 2020-12-26 17:10
start learning 2 2020-12-26 17:10... end learning 2020-12-26 17:19
start learning 3 2020-12-26 17:19... end learning 2020-12-26 17:27
start learning 4 2020-12-26 17:27... end learning 2020-12-26 17:36
start learning 5 2020-12-26 17:36... end learning 2020-12-26 17:45
2_moneyness ['145.91984451301983', [146.36918275525795, 142.3172888664879, 145.6955194750355, 149.03566289670448, 146.1815685716133]]
start learning 1 2020-12-26 17:45... end learning 2020-12-26 17:53

KeyboardInterrupt: ignored

In [ ]:
tag='featureCreation, rf 700/26/2'
LibUtils.WriteNote(tag, '1_base', '[155.8104417496264, [156.93994273062205, 158.2932151665893, 158.90776386933226, 152.36568233156436, 152.54560465002396]]')
LibUtils.WriteNote(tag, '2_moneyness', '[145.91984451301983, [146.36918275525795, 142.3172888664879, 145.6955194750355, 149.03566289670448, 146.1815685716133]]')
LibUtils.WriteNote(tag, '3_AttainedAge', '[148.69195052291712, [150.81654169236418, 144.5225177971377, 148.7017304332789, 151.18424651622334, 148.23471617558144]]')
LibUtils.WriteNote(tag, '4_Moneyness+AttainedAge', '[138.1263707880174, [137.43394975273944, 137.7690322930578, 134.97723476712014, 143.87908994542667, 136.57254718174292]]')

tag='featureCreation, rf 1000/30/1'
LibUtils.WriteNote(tag, '1_base', '[150.23484263937553, [153.54810761882993, 152.92207607698916, 149.29856155415732, 149.38784357133437, 146.01762437556678]]')
LibUtils.WriteNote(tag, '2_moneyness', '[141.34573818264795, [140.828113158679, 142.09079861379632, 141.08947067824568, 143.89128247801295, 138.82902598450576]]')
LibUtils.WriteNote(tag, '3_AttainedAge', '[142.92968353490784, [141.06436699260925, 144.8715332782952, 141.94263394305972, 141.1555789683605, 145.61430449221447]]')
LibUtils.WriteNote(tag, '4_Moneyness+AttainedAge', '[132.96734449347812, [134.99847312774355, 134.32149927071544, 131.14448336417655, 129.9937825348891, 134.3784841698659]]')
